In [1]:
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from collections import defaultdict
from gensim import corpora, models
from nltk.corpus import stopwords
from pprint import pprint


/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
data = pd.read_csv("data_cleaned/combined.csv")
data.head()

,id,title,format,affiliation 1,affiliation 2,affiliation 3,affiliation 4,affiliation 5,affiliation 6,affiliation 7,...,Abstract,lower_abstract,affiliation,mode of presentation,affiliation 8,affiliation 9,affiliation 10,tags (primary),tags (secondary),affiliation 11
0,2013-1,Increasing the Effectiveness of Your Scholarly...,preconference workshop,University of Utah,Tecker International,NaN,NaN,NaN,NaN,NaN,...,This workshop is designed for those that are b...,this workshop is designed for those that are b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-2,"Planning, Assessing, and Communicating Library...",preconference workshop,University of Illinois at Urbana-Champaign,Pierce College,NaN,NaN,NaN,NaN,NaN,...,Libraries in higher education are increasingly...,libraries in higher education are increasingly...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-3,"Plugged into User Behavior: Low-Budget, High-I...",preconference workshop,University of Texas Arlington,NaN,NaN,NaN,NaN,NaN,NaN,...,How do students really use library subject gui...,how do students really use library subject gui...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-4,Rediscover the Joy of Working Together!: Posit...,preconference workshop,Kansas State University,Rockhurst University,University of Kansas,University of Iowa,NaN,NaN,NaN,...,Join this collaborative session to enhance you...,join this collaborative session to enhance you...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-5,Speculation to Litigation—Copyright and Climat...,preconference workshop,University of Louisville,Columbia University,Brigham Young University,Purdue University,NaN,NaN,NaN,...,Changes in digital technologies have produced ...,changes in digital technologies have produced ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Define function to remove rows with no abstract in a dataframe
def remove_no_abstract(data):
    data = data[data["lower_abstract"] != "no abstract"]
    data = data[data["lower_abstract"] != "No abstract"]
    data = data[data["lower_abstract"] != "no abstract available"]
    return data

In [8]:
# remove rows with no abstract in a dataframe
data = remove_no_abstract(data)

In [9]:
set(data["year"].tolist())

{2007, 2009, 2011, 2013, 2015, 2017, 2019, 2021}

In [10]:
documents = data["lower_abstract"].dropna().tolist()

In [11]:
stoplist = set(stopwords.words("english"))
stoplist.update(["library", "libraries"])
print(stoplist)

{'be', "aren't", 'had', 'own', "haven't", "don't", "shan't", 'or', "hadn't", 'such', 'whom', 'has', 'her', 'doing', 'wasn', 'i', 'hasn', 'any', 'ours', "weren't", 'that', 'library', 'same', 'during', 'out', 'we', 're', "needn't", 'your', 'hadn', 'shan', 'being', 'now', "that'll", 'mightn', "doesn't", 'where', "shouldn't", 'are', 'into', 'very', 'can', 'his', 'through', "wasn't", 'our', 'am', 'once', "you're", 'themselves', "won't", 'd', 'she', 't', 'isn', 'while', 'too', 'above', 'don', 'all', 'between', 'what', 'other', 'm', 'with', 'its', 'below', "you've", 'is', 'does', "couldn't", 'theirs', 'off', 'ma', 'after', 'before', 've', 'the', 'he', 'some', 'shouldn', 'until', 'my', 'so', 'a', 'needn', 'most', 'down', 'as', 'did', 'have', 'up', 'won', 'herself', 'if', 'about', 'on', 'them', 'against', 'to', 'their', 'why', 'didn', 'than', "isn't", 'll', "mightn't", 'over', 'y', 'doesn', "mustn't", 'who', 'because', 'me', 'been', 'there', 'then', 'yourself', 'of', 'each', 'couldn', 'itself',

In [12]:
texts = [[word for word in document.lower().split() if word not in stoplist]
            for document in documents]

In [13]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [14]:
texts = [[token for token in text if frequency[token] > 1]
            for text in texts]

In [15]:
pprint(texts[:1])

[['workshop',
  'designed',
  'building',
  'programs',
  'institutions',
  'need',
  'strengthen',
  'revitalize',
  'local',
  'efforts.',
  'emphasis',
  'placed',
  'think',
  'strategically',
  'local',
  'programs',
  'develop',
  'skills',
  'influential',
  'without',
  'necessarily',
  'positional',
  'authority.',
  'workshop',
  'encourage',
  'participant',
  'interaction',
  'incorporating',
  'hands-on',
  'activities.']]


In [16]:
dictionary = corpora.Dictionary(texts)
dictionary.save('acrl.dict')

In [17]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('acrl.mm', corpus)

In [18]:
hdp = models.HdpModel(corpus, id2word=dictionary)

In [19]:
hdp.print_topics()

[(0,
  '0.006*students + 0.005*learning + 0.005*research + 0.005*librarians + 0.004*information + 0.004*academic + 0.003*student + 0.003*learn + 0.003*new + 0.003*literacy'),
 (1,
  '0.006*librarians + 0.006*information + 0.005*research + 0.005*academic + 0.004*students + 0.004*new + 0.004*learning + 0.003*learn + 0.003*literacy + 0.003*student'),
 (2,
  '0.005*librarians + 0.005*research + 0.004*academic + 0.004*information + 0.003*students + 0.003*literacy + 0.003*student + 0.003*learn + 0.003*assessment + 0.003*faculty'),
 (3,
  '0.003*academic + 0.003*information + 0.003*new + 0.003*librarians + 0.002*literacy + 0.002*students + 0.002*research + 0.002*student + 0.002*also + 0.002*university'),
 (4,
  '0.004*information + 0.004*librarians + 0.003*students + 0.003*academic + 0.003*research + 0.003*learning + 0.002*learn + 0.002*faculty + 0.002*literacy + 0.002*student'),
 (5,
  '0.002*research + 0.002*university + 0.002*students + 0.002*librarians + 0.001*information + 0.001*academic

In [20]:
pyLDAvis.enable_notebook()

In [21]:
gensimvis.prepare(hdp, corpus, dictionary)

/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from im

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.141421  0.004817       1        1  26.325688
1      0.142762 -0.007733       2        1  24.996366
2      0.133149 -0.002832       3        1  19.424707
3      0.096931  0.000250       4        1   4.509365
4      0.096593  0.011419       5        1   4.060353
...         ...       ...     ...      ...        ...
142   -0.022572  0.049513     146        1   0.001059
129   -0.023502  0.002007     147        1   0.001058
139   -0.023439 -0.006680     148        1   0.000985
144   -0.020752  0.043372     149        1   0.000912
137   -0.025704  0.002538     150        1   0.000728

[150 rows x 5 columns], topic_info=              Term        Freq       Total  Category  logprob  loglift
75      librarians  413.000000  413.000000   Default  30.0000  30.0000
183       research  384.000000  384.000000   Default  29.0000  29.0000
396       academic  352.000000  352.000000   Default  28.0000  28.0000
172    information  377.000000  377.000000   Default  27.0000  27.0000
83        students  362.000000  362.000000   Default  26.0000  26.0000
...            ...         ...         ...       ...      ...      ...
156   technologies    0.000723   24.386816  Topic150  -6.9116   1.4044
6725     examined.    0.000558   11.987411  Topic150  -7.1700   1.8562
671      concepts.    0.000519   10.498405  Topic150  -7.2430   1.9158
4930      imposter    0.000529   11.789322  Topic150  -7.2235   1.8194
4196         jobs,    0.000545   19.238489  Topic150  -7.1948   1.3584

[8436 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
489       1  0.069746           &
489       2  0.139492           &
489       3  0.488223           &
489       4  0.069746           &
489       5  0.069746           &
...     ...       ...         ...
6964      5  0.121163          ﬁt
7145      1  0.234575    ﬁve-year
7145      2  0.117287    ﬁve-year
7145      3  0.234575    ﬁve-year
7106      2  0.421797  ﬂexibility

[13470 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 10, 6, 7, 8, 9, 11, 12, 13, 14, 17, 21, 16, 23, 19, 22, 15, 20, 37, 32, 30, 29, 18, 26, 33, 28, 25, 34, 27, 54, 41, 35, 51, 56, 39, 31, 47, 40, 49, 72, 45, 73, 36, 42, 24, 46, 69, 44, 104, 53, 92, 67, 38, 58, 48, 64, 114, 125, 60, 105, 144, 111, 52, 85, 110, 76, 90, 61, 126, 63, 71, 82, 94, 55, 50, 62, 78, 109, 43, 57, 141, 139, 113, 133, 68, 88, 106, 96, 75, 95, 127, 123, 122, 98, 115, 83, 93, 84, 120, 129, 128, 103, 134, 87, 121, 136, 131, 116, 70, 91, 124, 108, 132, 86, 137, 89, 117, 147, 107, 149, 100, 102, 142, 112, 65, 77, 59, 74, 81, 80, 101, 66, 99, 146, 118, 97, 79, 119, 150, 148, 135, 143, 130, 140, 145, 138])

In [23]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=30)

In [24]:
lda.print_topics()

[(14,
  '0.009*"liaison" + 0.007*"librarians" + 0.007*"learn" + 0.007*"academic" + 0.006*"practice" + 0.005*"information" + 0.005*"mindfulness" + 0.005*"teaching" + 0.005*"including" + 0.005*"instruction"'),
 (0,
  '0.012*"staff" + 0.010*"librarians" + 0.008*"career" + 0.007*"e-resource" + 0.007*"provide" + 0.006*"strategies" + 0.006*"one" + 0.006*"turned" + 0.006*"mental" + 0.006*"learn"'),
 (18,
  '0.016*"information" + 0.011*"instruction" + 0.009*"grow" + 0.009*"literacy" + 0.008*"scholarly" + 0.007*"students" + 0.007*"automated" + 0.007*"communications." + 0.007*"program" + 0.007*"new"'),
 (6,
  '0.011*"program" + 0.010*"librarians" + 0.009*"community" + 0.008*"diversity" + 0.008*"new" + 0.006*"university" + 0.006*"academic" + 0.006*"color" + 0.006*"session" + 0.005*"learn"'),
 (11,
  '0.014*"research" + 0.011*"students" + 0.009*"librarians" + 0.009*"learning" + 0.008*"academic" + 0.008*"online" + 0.008*"learn" + 0.007*"open" + 0.007*"information" + 0.007*"scholarly"'),
 (7,
  '0.0

In [25]:
gensimvis.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
24     0.088004  0.002336       1        1  9.937932
7      0.054167  0.041930       2        1  7.570531
2      0.043000  0.006541       3        1  6.702217
11     0.065105  0.021651       4        1  5.442461
9      0.047054  0.057532       5        1  4.713153
14     0.039826  0.031732       6        1  4.487577
22    -0.022631  0.064100       7        1  4.103545
8      0.041153 -0.019776       8        1  3.979742
4      0.057909 -0.017980       9        1  3.818807
1      0.090163 -0.091116      10        1  3.647450
17     0.012367  0.050121      11        1  3.461956
13    -0.003236  0.092423      12        1  3.267287
27     0.010873  0.021978      13        1  3.154941
5      0.028985  0.002021      14        1  3.019915
21     0.071328 -0.047853      15        1  3.003046
10     0.010968  0.060729      16        1  2.831355
26    -0.008507  0.026016      17        1  2.804918
6     -0.020214 -0.013402      18        1  2.659615
3     -0.022899  0.001902      19        1  2.510168
28    -0.025671 -0.013056      20        1  2.303723
20    -0.034540  0.058682      21        1  2.202565
19    -0.026607 -0.023923      22        1  1.825064
0     -0.080239 -0.011522      23        1  1.744867
25    -0.053306 -0.027925      24        1  1.727086
12    -0.049937 -0.065370      25        1  1.688653
18     0.008819 -0.131934      26        1  1.665437
29    -0.092342 -0.019816      27        1  1.603359
16    -0.041356 -0.075606      28        1  1.533432
15    -0.067343 -0.010135      29        1  1.484860
23    -0.120892  0.029717      30        1  1.104338, topic_info=             Term        Freq       Total Category  logprob  loglift
138       digital  349.000000  349.000000  Default  30.0000  30.0000
172   information  895.000000  895.000000  Default  29.0000  29.0000
30     assessment  314.000000  314.000000  Default  28.0000  28.0000
174      literacy  535.000000  535.000000  Default  27.0000  27.0000
195          data  418.000000  418.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
254        social    4.821688  179.453360  Topic30  -5.4310   0.8891
712         share    4.934219  268.214954  Topic30  -5.4080   0.5103
242       discuss    4.947668  299.438002  Topic30  -5.4052   0.4029
4801    privilege    3.976732   10.437824  Topic30  -5.6237   3.5409
412           new    4.406978  491.443775  Topic30  -5.5210  -0.2082

[2056 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
489       2  0.095175           &
489       3  0.031725           &
489       5  0.095175           &
489       6  0.031725           &
489       8  0.222075           &
...     ...       ...         ...
2837      1  0.893254  “feminine”
6412     20  0.765758     “global
4534     11  0.762301        “how
3548     10  0.844921      “maker
5966     12  0.779093      “state

[7973 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[25, 8, 3, 12, 10, 15, 23, 9, 5, 2, 18, 14, 28, 6, 22, 11, 27, 7, 4, 29, 21, 20, 1, 26, 13, 19, 30, 17, 16, 24])